In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
tushar5harma_plant_village_dataset_updated_path = kagglehub.dataset_download('tushar5harma/plant-village-dataset-updated')

print('Data source import complete.')


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import preprocess_input
import json
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
import tensorflow as tf

# Load the validation dataset
validation_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  # Adjusting the image size to match DenseNet121 input
    shuffle=True,
    interpolation="bilinear",
    subset="validation",
    validation_split=0.2,  # Split data into training and validation
    seed=42  # Provide a seed for reproducibility
)


In [ ]:
import tensorflow as tf

# Load the training dataset
training_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  # Adjusting the image size to match DenseNet121 input
    shuffle=True,
    interpolation="bilinear",
    subset="training",
    validation_split=0.2,  # Split data into training and validation
    seed=42  # Provide a seed for reproducibility
)


In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input

# Preprocess the datasets to match DenseNet121 input requirements
training_set = training_set.map(lambda x, y: (preprocess_input(x), y))
validation_set = validation_set.map(lambda x, y: (preprocess_input(x), y))


In [ ]:
from tensorflow.keras.applications import DenseNet121

# Load DenseNet121 model without the top classification layer, using ImageNet weights
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
# Freeze the base model layers to prevent them from being trained
base_model.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import DenseNet121

# Load DenseNet121 base model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the new model on top of the DenseNet121 base
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')  # Adjust this according to your dataset classes
])


In [ ]:
from tensorflow.keras.metrics import Precision, Recall, AUC

# Compile the DenseNet121 model with additional metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',           # Standard accuracy
        Precision(),          # Precision metric
        Recall(),             # Recall metric
        AUC(),                # AUC (Area Under Curve)
        'TruePositives',      # True positive count
        'TrueNegatives',      # True negative count
        'FalsePositives',     # False positive count
        'FalseNegatives'      # False negative count
    ]
)


In [ ]:
# Model summary
model.summary()

In [ ]:
# Training the model
training_history = model.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=10
)


In [ ]:
# Evaluate the model on the training set
results = model.evaluate(training_set)

# Extracting loss and accuracy along with additional metrics
train_loss = results[0]
train_acc = results[1]
precision = results[2]
recall = results[3]
auc = results[4]
true_positives = results[5]
true_negatives = results[6]
false_positives = results[7]
false_negatives = results[8]

# Printing the results
print(f'Train Loss: {train_loss}, Train Accuracy: {train_acc}')
print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')


In [ ]:
# Evaluate the model on the validation set
results = model.evaluate(validation_set)

# Extracting loss and accuracy along with additional metrics
val_loss = results[0]
val_acc = results[1]
precision = results[2]
recall = results[3]
auc = results[4]
true_positives = results[5]
true_negatives = results[6]
false_positives = results[7]
false_negatives = results[8]

# Printing the results
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')
print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')


In [ ]:
import tensorflow as tf

# Load the test dataset from the 'Test' subdirectories of the plant categories
test_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  # Adjusting the image size to match DenseNet121 input
    shuffle=False,  # Typically, you don't shuffle the test set
    interpolation="bilinear",
)

# Evaluate the model on the test set
test_results = model.evaluate(test_set)

# Extracting test metrics
test_loss = test_results[0]
test_acc = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]
test_auc = test_results[4]
test_true_positives = test_results[5]
test_true_negatives = test_results[6]
test_false_positives = test_results[7]
test_false_negatives = test_results[8]

# Printing the results for the test set
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')
print(f'Test Precision: {test_precision}, Test Recall: {test_recall}, Test AUC: {test_auc}')
print(f'Test True Positives: {test_true_positives}, Test True Negatives: {test_true_negatives}')
print(f'Test False Positives: {test_false_positives}, Test False Negatives: {test_false_negatives}')


In [ ]:
# Save the DenseNet-121 model
model.save('plant-village_disease_model_densenet121.keras')


In [ ]:
# Save training history to JSON
with open('plant-village_densenet121_hist.json', 'w') as f:
    json.dump(training_history.history, f)
